In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K

# Sentence pairs (labels: 1 for similar, 0 for dissimilar)
sentence_pairs = [
    ("How are you?", "How do you do?", 1),
    ("How are you?", "What is your name?", 0),
    ("What time is it?", "Can you tell me the time?", 1),
    ("What is your name?", "Tell me the time?", 0),
    ("Hello there!", "Hi!", 1),
]

# Separate into two sets of sentences and their labels
sentences1 = [pair[0] for pair in sentence_pairs]
sentences2 = [pair[1] for pair in sentence_pairs]
labels = np.array([pair[2] for pair in sentence_pairs])

# Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences1 + sentences2)
vocab_size = len(tokenizer.word_index) + 1

# Convert sentences to sequences
max_len = 100  # Max sequence length
X1 = pad_sequences(tokenizer.texts_to_sequences(sentences1), maxlen=max_len)
X2 = pad_sequences(tokenizer.texts_to_sequences(sentences2), maxlen=max_len)

# Input layers for two sentences
input_1 = Input(shape=(max_len,))
input_2 = Input(shape=(max_len,))

# Embedding layer
embedding_dim = 1000
embedding = Embedding(vocab_size, embedding_dim, input_length=max_len)

# Shared LSTM layer
shared_lstm = Bidirectional(LSTM(512))

# Process the two inputs using the shared LSTM
encoded_1 = shared_lstm(embedding(input_1))
encoded_2 = shared_lstm(embedding(input_2))

# Calculate the L1 distance between the two encoded sentences
def l1_distance(vectors):
    x, y = vectors
    return K.abs(x - y)

l1_layer = Lambda(l1_distance)
l1_distance_output = l1_layer([encoded_1, encoded_2])

# Add a dense layer for classification (similar/dissimilar)
output = Dense(1, activation='sigmoid')(l1_distance_output)

# Create the Siamese network model
siamese_network = Model([input_1, input_2], output)
siamese_network.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
siamese_network.summary()

# Train the model
siamese_network.fit([X1, X2], labels, epochs=12, batch_size=2)

# Test with a new sentence pair
test_sentences1 = ["How are you?"]
test_sentences2 = ["How do you do?"]

test_X1 = pad_sequences(tokenizer.texts_to_sequences(test_sentences1), maxlen=max_len)
test_X2 = pad_sequences(tokenizer.texts_to_sequences(test_sentences2), maxlen=max_len)

# Predict similarity
similarity = siamese_network.predict([test_X1, test_X2])
print(f"Similarity Score: {similarity[0][0]}")


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_5             │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 100, 1000)      │         18,000 │ input_layer_4[0][0],   │
│                           │                        │                │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_2           │ (None, 1024)           │      6,197,248 │ embedding_2[0][0],     │
│ (Bidirectional)           │                        │                │ embedding_2[1][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_2 (Lambda)         │ (None, 1024)           │              0 │ bidirectional_2[0][0], │
│                           │                        │                │ bidirectional_2[1][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 1)              │          1,025 │ lambda_2[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 6,216,273 (23.71 MB)

 Trainable params: 6,216,273 (23.71 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 34s 8s/step - accuracy: 0.6125 - loss: 0.6932
Epoch 2/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 39s 7s/step - accuracy: 1.0000 - loss: 0.6358
Epoch 3/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 42s 7s/step - accuracy: 1.0000 - loss: 0.5526
Epoch 4/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 41s 7s/step - accuracy: 1.0000 - loss: 0.4521
Epoch 5/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 40s 7s/step - accuracy: 1.0000 - loss: 0.3458
Epoch 6/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 43s 7s/step - accuracy: 1.0000 - loss: 0.2831
Epoch 7/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 24s 7s/step - accuracy: 1.0000 - loss: 0.1441
Epoch 8/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 26s 8s/step - accuracy: 1.0000 - loss: 0.1454
Epoch 9/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 23s 7s/step - accuracy: 1.0000 - loss: 0.0814
Epoch 10/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 43s 7s/step - accuracy: 1.0000 - loss: 0.0398
Epoch 11/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 40s 8s/step - accuracy: 1.0000 - loss: 0.0133
Epoch 12/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 25s 7s/step - accuracy: 1.0000 - loss: 0.0074
1/1 ━━━━━━━━━